In [1]:
import numpy as np
import pandas as pd

In [2]:
def aggregateCluster(clusterNum,ptype):
    GLList=[]
    if ptype =='Batters':
        Clusters = pd.read_csv('./BatterClusters/PredBattingClusters.csv', index_col=0)
    else:
        Clusters = pd.read_csv('./PitcherClusters/PredPitchingClusters.csv', index_col=0)
    
    PlayersCluster = Clusters[Clusters['Labels']==clusterNum]
    for players in PlayersCluster.values:
        ID=players[1]
        if players[3] == 'Y':
            GameLog = pd.read_csv('../Data/DataBaseCrawler/GameLogs/'+ptype+'/'+str(ID)+'GL.csv', index_col=0)
        else:
            GameLog = pd.read_csv('../Data/DataBaseCrawler/GameLogs/Inactive'+ptype+'/'+str(ID)+'GL.csv', index_col=0)
        GameLog = GameLog[GameLog['Date']!='Total']
        GLList.append(GameLog)
    return pd.concat(GLList)

In [3]:
def stripAt(x):
    return x.strip('@')

In [4]:
def cleanGL(x):
    try:
        out = float(x)
    except:
        try:
            out = float(x.strip('%'))
        except:
            out = 0
    return out

In [5]:
BattingClusters = pd.read_csv('./BatterClusters/PredBattingClusters.csv', index_col=0)
PitchingClusters = pd.read_csv('./PitcherClusters/TotalPitchingClusters.csv', index_col=0)
NumBattingClusters = 32
NumPitchingClusterNum = 8

In [6]:
for i in range(NumBattingClusters):
    #Make the Dataframe
    ClusterSplitData=[]
    #Get the Battercluster
    BatterCluster = aggregateCluster(i,'Batters')
    BatterCluster = BatterCluster[~BatterCluster['Pos'].isin(['PR-LF', 'PH-CF', 'PH-RF','PH-1B','PH-SS','PH-2B','PR-C','P','PH-RF-CF','PR-DH','PH-3B','PH-C','PH','PH-LF','DH','PR-1B','PR-RF','PH-DH','1B-PH','RF-PH'])]
    BatterCluster = BatterCluster[BatterCluster['BO']!=0]
    BatterCluster['Opp']=BatterCluster['Opp'].apply(stripAt)
    for j in range(NumPitchingClusterNum):
        #Get the Pitcher Cluster
        PitcherCluster=aggregateCluster(j,'Pitchers')
        PitcherCluster=PitcherCluster[PitcherCluster['GS']==1]
        PitcherCluster['Opp']=PitcherCluster['Opp'].apply(stripAt)
        
        #Get Cluster
        MergedClusters=pd.merge(BatterCluster,PitcherCluster, left_on=['Date','Team','Opp'], right_on=['Date','Opp','Team'])
        MergedClusters=MergedClusters[MergedClusters['PA']!=0]
        
        #Clean the merge.
        MergedClusters=MergedClusters[['BO', 'G_x', 'AB', 'PA', 'H_x', '1B','2B', '3B', 'HR_x', 'R_x','SO_x','HBP_x', 'BB_x', 'IBB_x','RBI','AVG_x','DKfp_x','SF', 'SH', 'SB', 'CS', 'BB%_x', 'K%_x','BB/K', 'OBP', 'SLG', 'OPS', 'ISO', 'Spd', 'BABIP_x', 'wSB', 'wRC','wRAA', 'wOBA', 'wRC+']].applymap(cleanGL)
        
        #Get the count.
        n, d = MergedClusters.shape
        
        #Get the sum
        ClusterSplit = MergedClusters.sum(axis=0).to_frame().T
        
        #Normalize the split to PA
        PA = ClusterSplit.values[0][3]
        G = ClusterSplit.values[0][1]
        
        #Normalize by plate apperance
        NormalizedValues = [x/G for x in ClusterSplit.values[0][0:1]]+[x for x in ClusterSplit.values[0][1:4]]+[x/PA for x in ClusterSplit.values[0][4:14]]+[x/G for x in ClusterSplit.values[0][14:]]
        
        #Put the row in the clustersplit
        ClusterSplitData.append(NormalizedValues)
        
    pd.DataFrame(data=ClusterSplitData, columns=MergedClusters.columns).to_csv('./ClusterSplits/BatterSplit'+str(i)+'.csv')